In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Carga los logs de la carpeta weblogs

In [11]:
df = spark.read.text("weblogs/2014-01-18.log")
df.printSchema()

root
 |-- value: string (nullable = true)



In [2]:
df.describe().show()

root
 |-- value: string (nullable = true)

+-------+--------------------+
|summary|               value|
+-------+--------------------+
|  count|                6403|
|   mean|                null|
| stddev|                null|
|    min|1.119.102.62 - 83...|
|    max|99.73.191.11 - 20...|
+-------+--------------------+



In [12]:
logs=sc.textFile("weblogs/*.log")
logs.take(1)

['116.180.70.237 - 128 [15/Sep/2013:23:59:53 +0100] "GET /hive-00031.html HTTP/1.0" 200 1388 "http://www.cursoDeFormacionBigData.com"  "Opera 3.4"']

2. Crea un RDD de pares a partir de estos ficheros con el usuario como clave y un valor 1

In [4]:
usuariosRDD = logs.map(lambda linea: linea.split()).map(lambda words: (words[2].strip(),1))
usuariosRDD.take(5)

[('128', 1), ('128', 1), ('94', 1), ('94', 1), ('131', 1)]

3. Crea un nuevo RDD con la suma de los valores para cada identificador de usuario

In [5]:
usuariosTotal = usuariosRDD.reduceByKey(lambda a,b: a+b)
usuariosTotal.take(5)

[('54126', 32), ('54', 498), ('21443', 35), ('74', 516), ('64', 562)]

4. Muestra los 10 usuarios que más peticiones han realizado.

In [7]:
usuariosTotal.takeOrdered(10, lambda x: -x[1])

[('193', 667),
 ('13', 636),
 ('24', 620),
 ('147', 610),
 ('110', 606),
 ('84', 606),
 ('111', 604),
 ('77', 603),
 ('21', 600),
 ('126', 600)]

In [10]:
usuariosTotal.sortBy(lambda x: -x[1]).take(10)

[('193', 667),
 ('13', 636),
 ('24', 620),
 ('147', 610),
 ('110', 606),
 ('84', 606),
 ('111', 604),
 ('77', 603),
 ('21', 600),
 ('126', 600)]

5. Crea un nuevo RDD de pares donde la clave sea el código del usuario y el valor una lista de IP desde donde se ha conectado el usuario.  

In [13]:
usuariosIP = logs.map(lambda linea: linea.split()).map(lambda words: (words[2].strip(),words[0].strip()))

In [14]:
usuariosIP.take(3)

[('128', '116.180.70.237'),
 ('128', '116.180.70.237'),
 ('94', '218.193.16.244')]

In [18]:
usuariosIP.groupByKey().map(lambda x: (x[0], list(x[1]))).take(1)

[('54126',
  ['176.80.85.20',
   '176.80.85.20',
   '216.249.102.215',
   '216.249.102.215',
   '189.97.51.33',
   '189.97.51.33',
   '191.88.233.113',
   '191.88.233.113',
   '191.88.233.113',
   '191.88.233.113',
   '161.94.252.10',
   '161.94.252.10',
   '69.103.100.120',
   '69.103.100.120',
   '31.252.214.103',
   '31.252.214.103',
   '244.131.187.64',
   '244.131.187.64',
   '220.61.164.139',
   '220.61.164.139',
   '17.171.169.4',
   '17.171.169.4',
   '17.171.169.4',
   '17.171.169.4',
   '75.13.247.212',
   '75.13.247.212',
   '104.14.31.172',
   '104.14.31.172',
   '224.190.37.8',
   '224.190.37.8',
   '211.209.98.155',
   '211.209.98.155'])]

In [19]:
# Sin repeticiones
usuariosIP.groupByKey().map(lambda x: (x[0], set(x[1]))).take(1)

[('54126',
  {'104.14.31.172',
   '161.94.252.10',
   '17.171.169.4',
   '176.80.85.20',
   '189.97.51.33',
   '191.88.233.113',
   '211.209.98.155',
   '216.249.102.215',
   '220.61.164.139',
   '224.190.37.8',
   '244.131.187.64',
   '31.252.214.103',
   '69.103.100.120',
   '75.13.247.212'})]

## Contando tipos de formatos


In [37]:
# rsplit empieza por la derecha, y le decimos que queremos que solo parta una vez,
# de manera que devolverá 2 elementos, y nos quedamos con el 2º, que es lo que está a la derecha del punto
formatosRDD = logs.map(lambda linea: linea.split()).map(lambda words: (words[6].rsplit(".", 1)[1], 1))

In [38]:
formatosRDD.reduceByKey(lambda x,y: x+y).collect()

[('js', 25212),
 ('html', 173049),
 ('png', 4357),
 ('jpg', 24743),
 ('txt', 6096),
 ('css', 180632)]